МФТИ ФИВТ, Машинное обучение, Никита Волков

# Ранжирование

* Дедлайн **14 декабря 23:59** для всех групп.
* В качестве решения задания нужно прислать ноутбук с подробными комментариями.
* Кагл: https://inclass.kaggle.com/c/ml-mipt-ranking4
* Инвайт: https://kaggle.com/join/4uthnui9sndi

### Небольшая предыстория

Внезапно очутившись на 4 курсе (как это произошло, вообще не понятно) студенты ФИВТа поняли, что времени стало совсем не хватать. Немного подумав, они решили, что во всем виноваты обезумевшие преподаватели курса Машинное обучение. Было проведено коллективное собрание, на котором было принято решение — попробовать договориться с преподавателями. Однако, все попытки примирения были отвергнуты. И тогда на очередном собрании (вместо выполнения очередного ДЗ ;)) было принято решение сделать вопросно-ответную систему для студентов. Например, она сможет четко отвечать на вопросы следующего типа ”Когда дедлайн по очередному огромному заданию?”, ”Что нужно сделать в задании?”, а вопрос ”Сколько суток нужно потратить, чтобы выполнить очередное ДЗ” даже никто и задавать не будет — никто их не делает, кроме самих преподавателей. А на вопрос ”Сможем ли мы переубедить Арсения Ашуху?” система ответит картинкой:

<img width=400 src="./pic.png">

Для каждого вопроса студенты смогли написать систему, которая подбирает наиболее подходящие ответы в интернете. А вот, как из упорядочить, они не знают, поскольку прогуляли лекцию по ранжированию. К счастью, было записано видео этой лекции, так что вы сможете им помочь.

А теперь серьезно...

### Формат файлов

Вам выдается 4 файла:

* `train.txt` --- обучающая выборка пар запрос-документ и асессорские метки релевантности
* `test.txt` --- тестовая выборка пар запрос-документ
* `queries_test.txt` --- запросы из `train.txt`
* `queries_train.txt` --- запросы из `test.txt`

Колонки в файлах могут быть следующего типа:

* `QueryId` --- уникальный номер запроса
* `DocumentId` --- номер документа, не повторяется для одного запроса
* `DocumentLink` --- url документа
* `Relevance` --- асессорская метка релевантности

Формат файла ответов приведен ниже. Пары запрос-документ должны соответсвовать файлу `test.txt` и должны быть упорядочены по убыванию построенной функции релевантности. То есть так, как в поисковой выдаче.

Качество оценивается с помощью метрики ${NDCG}_{10}$.

### Бейзлайн

In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen, HTTPError
import numpy as np

Считываем все запросы из test

In [2]:
with open('queries_test.txt') as f:
    queries = f.readlines()
queries = list(map(lambda x: x[:-1].split('\t')[1], queries))

По очереди загружаем все документы, считая для каждого TF и сортируя для каждого запроса. Это и есть скор в бейзлайне.

In [11]:
with open('test.txt') as ft:
    with open('baseline-1.txt', 'w') as fb:
        ft.readline()
        fb.write('QueryId,DocumentId\n')

        qids, docids, relevs = [], [], []
        doc_count = 0 # number of documents, relevant to one specific query (cardinality of D)
        doc_counts = []
        doc_w_count = dict() # cardinality of D_w for each w (word in query)
        doc_w_counts = []
        for line in ft:
            tmp = line.split(',')
            if len(tmp) != 3:
                qid, docid = tmp[:2]
                url = ','.join(tmp[2:])
#                 print(tmp)
#                 print(url)
            else:
                qid, docid, url = tmp
            qid, docid = int(qid), int(docid)
            if len(qids) > 0 and qid != qids[-1]:
                # output
#                 order = np.argsort(relevs)[::-1]
#                 qids = np.array(qids)[order]
#                 docids = np.array(docids)[order]
                
#                 for i in range(len(qids)):
#                     fb.write('{},{}\n'.format(qids[i], docids[i]))
                
                doc_counts.append(doc_count)
                doc_w_counts.append(doc_w_count)
                print('doc_count:', doc_count) # temporary
                print('doc_w_count:', doc_w_count) # temporary
                break # temporary
                qids, docids, relevs = [], [], []
                doc_count = 0
                doc_w_count = dict()
                print(qid)

            doc_count += 1
            try:
                text = ' '.join(BeautifulSoup(urlopen(url, timeout=10), 'lxml').stripped_strings)
            except Exception:
                text = '#' # treat such webpages as empty
                
            text = text.split(' ')
            text = np.array(list(filter(lambda x: x, text))) # filter empty strings

            term_freq = 0
            for word in filter(lambda x: x, queries[qid - 101].split(' ')):
                mask = text == word
                term_freq_word = mask.mean()
                doc_w_count[word] = doc_w_count.get(word, 0) + (1 if term_freq_word > 0 else 0)
                term_freq += term_freq_word
            print(term_freq)
            print(qid, ':', doc_w_count)
                
            qids.append(qid)
            docids.append(docid)
            relevs.append(term_freq)

0.0452488687783
101 : {'просто': 1, 'аккуратненько': 1, 'мужика': 1, 'нельзя': 1, 'вернуть': 1, 'тебе': 1, 'неподошедшего': 1, 'маме?': 1, 'Почему': 1, 'обратно': 1}
0.00570768934778
101 : {'просто': 2, 'аккуратненько': 1, 'мужика': 2, 'нельзя': 1, 'вернуть': 2, 'тебе': 2, 'неподошедшего': 1, 'маме?': 1, 'Почему': 2, 'обратно': 1}
0.000617093489664
101 : {'просто': 3, 'аккуратненько': 1, 'мужика': 2, 'нельзя': 2, 'вернуть': 2, 'тебе': 2, 'неподошедшего': 1, 'маме?': 1, 'Почему': 3, 'обратно': 1}
0.00240096038415
101 : {'просто': 4, 'аккуратненько': 1, 'мужика': 2, 'нельзя': 2, 'вернуть': 3, 'тебе': 3, 'неподошедшего': 1, 'маме?': 1, 'Почему': 3, 'обратно': 1}
0.00542888165038
101 : {'просто': 5, 'аккуратненько': 1, 'мужика': 2, 'нельзя': 3, 'вернуть': 4, 'тебе': 3, 'неподошедшего': 2, 'маме?': 1, 'Почему': 4, 'обратно': 1}
0.125
101 : {'просто': 6, 'аккуратненько': 2, 'мужика': 3, 'нельзя': 4, 'вернуть': 5, 'тебе': 4, 'неподошедшего': 3, 'маме?': 2, 'Почему': 5, 'обратно': 2}


KeyboardInterrupt: 

Если у вас не загружаются некоторые страницы, подумайте, как с этим можно бороться. Например, посмотрите код с семинара и из предыдущего ДЗ. Если и это не помогает, то нужно подумать, что делать в этой ситуации.

### Кажется, это последнее задание

Свобода!

<img width=200 src="./last.jpg">

А, нет, еще экзамен...